In [1]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pickle
import pandas as pd

#Plot settings
%matplotlib inline
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

#line thickness
import matplotlib as mpl
#mpl.rcParams['lines.linewidth'] = 5

from plotter import *

#Dataset parameters
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
    xs = json.load(f)

with open('pmap.json') as f:
    pmap = json.load(f)

systematics = ['nominal',
               'jet_triggerUp','jet_triggerDown',
               'btagWeightUp','btagWeightDown','btagEffStatUp','btagEffStatDown',
               'UESUp','UESDown','JESUp','JESDown','JERUp','JERDown',
              ]

mc = ['QCD','Wjets','Zjets','ttbar','singlet','VV','ggF','VBF','ZH','WH']


#To calculate significance
def significance(s,b):
    if b==0:
        return 0
    z_squared = 2.0*(s+b)*np.log(1.0+1.0*s/b) - 2.0*s
    return np.sqrt(z_squared)

## 1. Processing plots after Jennet's run

Coffea outputs: /eos/uscms/store/user/jennetd/may-2021/vh-charm-category/outfiles

In [2]:
year = '2017'
outsum = processor.dict_accumulator()
nfiles = len(subprocess.getoutput("ls ../infiles-split/"+year+"*.json").split())
#nfiles = 10

# Check if pickle exists, and don't re-create it if it does
repickle=True

picklename = 'pickles/{}_CScores_VHCharmSample_064.pkl'.format(year)
if os.path.isfile(picklename):
    repickle=False

In [3]:
# Load all files - this takes a while
if repickle:
    for n in range(1, nfiles+1):

        with open('../infiles-split/{}_{}.json'.format(year, n)) as f:
            infiles = json.load(f)
    
        filename = '/myeosdir/vh-charm-category/outfiles-064-ddb2/{}_{}.coffea'.format(year, n)
        #filename = 'outfiles/'+year+'_'+str(n)+'.coffea' #For local testing
        
        if os.path.isfile(filename):
            try:
                out = util.load(filename)
                outsum.add(out)
            except:
                print("File {} is broken.".format(filename))
        else:
            print('Missing file: '+str(n), infiles.keys())
            #print("File " + filename + " is missing")
        
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
    outsum['templates-vh-2'].scale(scale_lumi, 'dataset')
    
    # Use pmap to group the datasets together
    templates = outsum['templates-vh-2'].group('dataset', hist.Cat('process', 'Process'), pmap)

    outfile = open(picklename, 'wb')
    pickle.dump(templates, outfile, protocol=-1)
    outfile.close()

Missing file: 2416 dict_keys(['WJetsToLNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8'])


/opt/conda/lib/python3.8/site-packages/coffea/hist/hist_tools.py:376: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7f6f5bfcb6d0>
  warnings.warn(


In [4]:
# Read the histogram from the pickle file
templates = pickle.load(open(picklename,'rb')).sum('msd2','ddb2').integrate('msd1',slice(117.,131.)).integrate('ddb1',slice(0.64,1)).integrate('region','signal')

## See how many events are in each two processes

In [5]:
def event_in_process_larger(templates):
    
    #New label for columns
    cols = [">={}".format(round(x,1)) for x in templates.axis('ddc2').edges()]
    edge_vals = templates.axis('ddc2').edges() #Coresponding edge value to use later

    df = pd.DataFrame(index = mc, columns = cols)
    
    #loop over the processes and ddc2 scores and calcualte the events
    for process in mc:
        for thres in edge_vals:
            df.loc[process, ">={}".format(round(thres,1))] = round(templates.integrate('process', process).integrate('ddc2', slice(thres,1)).values()[()])
    
    return df

event_in_process_larger(templates)

,>=0.0,>=0.2,>=0.4,>=0.6,>=0.8,>=1.0
QCD,59,44,21,5,0,0
Wjets,1,0,0,0,0,0
Zjets,2,1,0,0,0,0
ttbar,6,5,5,3,2,0
singlet,1,1,1,1,0,0
VV,0,0,0,0,0,0
ggF,1,0,0,0,0,0
VBF,0,0,0,0,0,0
ZH,1,1,1,1,0,0
WH,2,2,1,1,0,0


In [6]:
def event_in_process_smaller(templates):
    
    #New label for columns
    cols = ["<{}".format(round(x,1)) for x in templates.axis('ddc2').edges()]
    edge_vals = templates.axis('ddc2').edges() #Coresponding edge value to use later

    df = pd.DataFrame(index = mc, columns = cols)
    
    #loop over the processes and ddc2 scores and calcualte the events
    for process in mc:
        for thres in edge_vals:
            df.loc[process, "<{}".format(round(thres,1))] = round(templates.integrate('process', process).integrate('ddc2', slice(0.,thres)).values()[()])
    
    return df

event_in_process_smaller(templates)

,<0.0,<0.2,<0.4,<0.6,<0.8,<1.0
QCD,15,15,38,55,59,59
Wjets,0,0,1,1,1,1
Zjets,1,1,1,1,2,2
ttbar,1,1,1,2,4,6
singlet,0,0,0,0,1,1
VV,0,0,0,0,0,0
ggF,1,1,1,1,1,1
VBF,0,0,0,0,0,0
ZH,0,0,0,1,1,1
WH,0,0,1,1,2,2


## ZH, WH processes significance

### Not scaled version

In [7]:
def significance_scaled(s,b):
    
    #s = s*137.3/41.5
    #b = b*137.3/41.5
    
    if b==0:
        return 0
    z_squared = 2.0*(s+b)*np.log(1.0+1.0*s/b) - 2.0*s
    return np.sqrt(z_squared)

def significance_table(templates, proc = 'ZH'):
    
    indx = ['>=', '<=', "Quadrature Sum"]
    cols = ["{}".format(round(x,1)) for x in templates.axis('ddc2').edges()]
    edge_vals = templates.axis('ddc2').edges()
    
    significance_df = pd.DataFrame(index = indx, columns = cols)
    
    b_list = [x for x in mc if x not in ['ZH', 'WH']] #Background
    
    for cut in indx:
        for thres in edge_vals:
            
            if cut == '>=': #Integrate forward
                #Calculate the number of signals and background
                s = templates.integrate('process', proc)\
                                        .integrate('ddc2', slice(thres,1)).values()[()]
                
                b = templates.integrate('process', b_list)\
                                        .integrate('ddc2', slice(thres,1)).values()[()]

                significance_df.loc[cut, "{}".format(round(thres,1))] = significance_scaled(s,b)
                
            else: #Integrate backwards
                #Calculate the number of signals and background
                s = templates.integrate('process', proc)\
                                        .integrate('ddc2', slice(0.,thres)).values()[()]
               
                b = templates.integrate('process', b_list)\
                                        .integrate('ddc2', slice(0.,thres)).values()[()]

                significance_df.loc[cut, "{}".format(round(thres,1))] = significance_scaled(s,b)
    
    def hypot(x):
        return np.hypot(x[0], x[1])

    significance_df.loc['Quadrature Sum'] = significance_df.apply(hypot, axis = 0)
    
    return significance_df


significance_table(templates)

,0.0,0.2,0.4,0.6,0.8,1.0
>=,0.149566,0.159153,0.185513,0.212244,0.109843,0
<=,0.024317,0.024317,0.042414,0.078309,0.132453,0.149566
Quadrature Sum,0.15153,0.161,0.1903,0.22623,0.172074,0.149566


In [8]:
significance_table(templates, "WH")

,0.0,0.2,0.4,0.6,0.8,1.0
>=,0.246356,0.269063,0.283229,0.320517,0.219241,0
<=,0.028041,0.028041,0.087707,0.139819,0.211058,0.246356
Quadrature Sum,0.247946,0.27052,0.296498,0.349686,0.304322,0.246356


### Scaled version

In [ ]:
def significance_scaled(s,b):
    
    s = s*137.3/41.5
    b = b*137.3/41.5
    
    if b==0:
        return 0
    z_squared = 2.0*(s+b)*np.log(1.0+1.0*s/b) - 2.0*s
    return np.sqrt(z_squared)

def significance_table(templates, proc = 'ZH'):
    
    indx = ['>=', '<=', "Quadrature Sum"]
    cols = ["{}".format(round(x,1)) for x in templates.axis('ddc2').edges()]
    edge_vals = templates.axis('ddc2').edges()
    
    significance_df = pd.DataFrame(index = indx, columns = cols)
    
    b_list = [x for x in mc if x not in ['ZH', 'WH']] #Background
    
    for cut in indx:
        for thres in edge_vals:
            
            if cut == '>=': #Integrate forward
                #Calculate the number of signals and background
                s = templates.integrate('process', proc)\
                                        .integrate('ddc2', slice(thres,1)).values()[()]
                
                b = templates.integrate('process', b_list)\
                                        .integrate('ddc2', slice(thres,1)).values()[()]

                significance_df.loc[cut, "{}".format(round(thres,1))] = significance_scaled(s,b)
                
            else: #Integrate backwards
                #Calculate the number of signals and background
                s = templates.integrate('process', proc)\
                                        .integrate('ddc2', slice(0.,thres)).values()[()]
               
                b = templates.integrate('process', b_list)\
                                        .integrate('ddc2', slice(0.,thres)).values()[()]

                significance_df.loc[cut, "{}".format(round(thres,1))] = significance_scaled(s,b)
    
    def hypot(x):
        return np.hypot(x[0], x[1])

    significance_df.loc['Quadrature Sum'] = significance_df.apply(hypot, axis = 0)
    
    return significance_df


significance_table(templates)

In [ ]:
significance_table(templates, "WH")